# Q42: AR Model

## Part 1

Consider the following model:

\begin{eqnarray}
A \sim & N(A; 0, 1.2) \\
R \sim & IG(R; 0.4, 250) \\
x_k|x_{k-1},A,R \sim & N(x_k; Ax_{k-1}, R) \\
x_0 = & 1 \quad x_1 = -6
\end{eqnarray}

### 1. Draw the directed graphical model and  the factor graph

##### Directed Graphical Model
<img src='graphs/DG1.png'>

##### Factor Graph
<img src='graphs/FG1.png'>

##### Directed Graphical Model and Factor Graph for AR(1) Model

<img src='graphs/DGFG.png'>

### 2. Write the expression for the full joint distribution and assign terms to the individuals factors on the graph

** Step 1: ** Write down the log of the full joint (unnormalised) posterior $log \phi = p(A, R, x_1=\hat{x}_1 | x_0=\hat{x}_0)$

$P=1.2, \nu=0.4, \beta=0.0016$

\begin{eqnarray}
\phi & = & p(A, R, x_1=\hat{x}_1 | x_0=\hat{x}_0) \propto p(x_1 | x_0, A, R) p(A) p(R) \\[7pt]
& = &  N(x_1; Ax_0, R) N(A; 0, P) IG(R, \nu, \nu/\beta) \\[7pt]
& \propto & exp\left( -\frac{1}{2} \frac{x_1^2}{R} + x_0 x_1 \frac{A}{R} - \frac{1}{2} \frac{x_0^2 A^2}{R} - \frac{1}{2} log 2\pi R \right) \\[7pt]
& & exp\left( -\frac{1}{2} \frac{A^2}{P} - \frac{1}{2} log|2 \pi P| \right) \\[7pt]
& & exp\left( -(\nu+1) logR - \frac{\nu}{\beta} \frac{1}{R} - log \Gamma(\nu) + \nu log(\nu/\beta) \right)
\end{eqnarray}


** Step 2: ** Assign each term of $log \phi$ to individual factors

\begin{eqnarray}
log \phi & = & -\frac{1}{2} \frac{x_1^2}{R} + x_0 x_1 \frac{A}{R} - \frac{1}{2} \frac{x_0^2 A^2}{R} - \frac{1}{2} log 2\pi R -\frac{1}{2} \frac{A^2}{P} - \frac{1}{2} log|2 \pi P| -(\nu+1) logR - \frac{\nu}{\beta} \frac{1}{R} - log \Gamma(\nu) + \nu log(\nu/\beta) \\[7pt]
& = & -\frac{1}{2} \frac{x_1^2}{R} + x_0 x_1 \frac{A}{R} - \frac{1}{2} \frac{x_0^2 A^2}{R} - \frac{1}{2} logR - \frac{1}{2} \frac{A^2}{P} -(\nu+1) logR - \frac{\nu}{\beta} \frac{1}{R} + C
\end{eqnarray}

Assignment of the factors:

\begin{eqnarray}
log\phi_A & = & -\frac{1}{2}\frac{A^2}{P} \\
log\phi_R & = &-(\nu+1+1/2)logR-(\frac{\nu}{\beta}+\frac{1}{2}x_1^2)\frac{1}{R} \\
log\phi_{x_{01}} & = & x_0x_1\frac{A}{R}-\frac{1}{2}\frac{x_0^2A^2}{R}
\end{eqnarray}


\\

<img src='graphs/FG2.png' style='width:600px'>


### 3. Derive the full conditional distributions $p(A|R,x_0,x_1)$ and $p(R|A,x_0,x_1)$

We can derive the full conditional distributions directly from the factor graph.

For $p(A|R,x_0,x_1)$ because R is given we can directly derive it from $\phi_A and \phi_{x_{01}}$

\begin{eqnarray}
p(A|R,x_0,x_1) & = & \phi_A \phi_{x_{01}} & = & exp\left(-\frac{1}{2}\frac{A^2}{P} + x_0x_1\frac{A}{R}-\frac{1}{2}\frac{x_0^2A^2}{R}\right)
\end{eqnarray}


For $p(R|A,x_0,x_1)$ because A is given we can directly derive it from $\phi_R and \phi_{x_{01}}$

\begin{eqnarray}
p(R|A,x_0,x_1) & = & \phi_R \phi_{x_{01}} & = & exp\left(-(\nu+1+1/2)logR-(\frac{\nu}{\beta}+\frac{1}{2}x_1^2)\frac{1}{R} + x_0x_1\frac{A}{R}-\frac{1}{2}\frac{x_0^2A^2}{R}\right)
\end{eqnarray}

### 4. Derive the joint distribution $p(A,R,x_0=1, x_1=-6)$ and create a contour plot.

We have already derived the full joint distribution, so we only need to put the values of $x_0$ and $x_1$ in the derivation.

$P=1.2, \nu=0.4, \beta=0.0016$

\begin{eqnarray}
p(A,R,x_0=1, x_1=-6) & \propto & N(x_1; Ax_0, R) N(A; 0, P) IG(R, \nu, \nu/\beta) \\[7pt]
& = & exp\left( -\frac{1}{2} \frac{x_1^2}{R} + x_0 x_1 \frac{A}{R} - \frac{1}{2} \frac{x_0^2 A^2}{R} - \frac{1}{2} log 2\pi R \right) \\[7pt]
& & exp\left( -\frac{1}{2} \frac{A^2}{P} - \frac{1}{2} log|2 \pi P| \right) \\[7pt]
& & exp\left( -(\nu+1) logR - \frac{\nu}{\beta} \frac{1}{R} - log \Gamma(\nu) + \nu log(\nu/\beta) \right) \\[7pt]
& = & exp\left( -\frac{18}{R} - 6\frac{A}{R} - \frac{1}{2} \frac{A^2}{R} - \frac{1}{2} log 2\pi R \right) \\[7pt]
& & exp\left( -\frac{A^2}{2.4} - \frac{1}{2} log|2.4 \pi| \right) \\[7pt]
& & exp\left( -(1.4) logR - 250\frac{1}{R} - log \Gamma(0.4) + 0.4 log(250) \right)
\end{eqnarray}



In [37]:
import matplotlib.pyplot as plt
import numpy as np

nu = 0.4
beta = 100
nu_beta = nu/beta

P = 1.2
x_0 = 1
x_1 = -6

T = 300 # Number of iterations
E_A = -6
E_A2 = E_A^2
E_invR = 1/0.00001 # Initial Sufficient stats

A = np.random.normal(0, P, 100)
R = nu_beta / np.random.gamma(shape=nu, size = 100)

for t in range(2,T):
    # Update q(A)
    Sig = 1/(1/P + x_0**2*E_invR)
    mu = Sig*x_0*x_1*E_invR
    E_A = mu
    E_A2 = mu**2 + Sig

    # Update q(R)
    a = nu+0.5
    b = 0.5*(x_1**2 - 2*x_1*x_0*E_A + x_0**2*E_A2) + nu_beta
    E_invR = a/b

print(E_A)
print(E_invR)

-0.3704347574553939
0.05483471954102366
